In [7]:
import pickle
import pandas as pd
import numpy as np
import importlib
from tqdm import tqdm
import time
import datetime
import h5py
import glob
import cv2
import scipy.signal

In [36]:
global main_data_path 
main_data_path = '/projects/EKOLEMEN/aza_lenny_data1/template/' 

all_shots = glob.glob('/scratch/gpfs/aj17/ece_co2/*.pkl') # This is just because we currently have CO2 data for some of the shots
all_shots = [np.int(x[x.rfind('/')+1:x.rfind('.')]) for x in all_shots]

In [13]:
def create_data_ece_co2(shotn,timing):
    # Load ECE signals, extract spectrograms and store in a dataframe
    spec_params={
        'nperseg': 512, # default 1024
        'noverlap': 256, # default: nperseg / 4
        'fs': 500000, # raw signal sample rate is 4MHz
        'window': 'hamm',
        'scaling': 'density', # {'density', 'spectrum'}
        'detrend': 'linear', # {'linear', 'constant', False}
        'eps': 1e-11,
        'enhanced':False # if True, the denoised spectrograms are stored
    }        
    all_data=dict()
    fl_lst = glob.glob(main_data_path+'%i_ece*.h5' % (shotn))
    if len(fl_lst) !=1:
        print('Error in loading ECE data!')
    else:
        print('ECE data found!')
        hf = h5py.File(fl_lst[0], 'r')
        sig_time = np.asarray(hf['ece']['xdata'])
        sig_time = np.where(np.logical_or(sig_time<timing['begin'],sig_time>timing['end']),np.nan,sig_time) # Cut everything beyond the timing['begin', 'end']
        for ece_idx in range(40):
            sig_data = np.asarray(hf['ece']['tecef%.2d' % (ece_idx+1)])
            sig_data = sig_data[np.logical_not(np.isnan(sig_time))]
            Sxx,f,t = specgr (sig_data,spec_params)
            all_data['ece%.2d' % (ece_idx+1)]= pd.DataFrame(Sxx.T,columns=['ece%.2d_%d' % (ece_idx+1,x+1) for x in range(np.int32(spec_params['nperseg']/2))],index=t)
        hf.close()
        all_data['ece_freq'] = f


    # Load CO2 signals, extract spectrograms and store in a dataframe
    # It is very similar to ECE but just different h5 key names

    spec_params={
        'nperseg': 1024, # default 1024
        'noverlap': 512, # default: nperseg / 4
        'fs': 1670000, # raw signal sample rate is 4MHz
        'window': 'hamm',
        'scaling': 'density', # {'density', 'spectrum'}
        'detrend': 'linear', # {'linear', 'constant', False}
        'eps': 1e-11,
        'enhanced':False # if True, the denoised spectrograms are stored
    }
    fl_lst = glob.glob(main_data_path+'%i_co2_pl.h5' % (shotn))
    if len(fl_lst) !=1:
        print('Error in loading CO2 data!')
    else:
        print('CO2 data found!')
        hf = h5py.File(fl_lst[0], 'r')
        sig_time = np.asarray(hf['co2_time'])
        sig_time = np.where(np.logical_or(sig_time<timing['begin'],sig_time>timing['end']),np.nan,sig_time)
        for co2_idx in ['r0','v1','v2','v3']:
            sig_data = np.asarray(hf['pl1%s_uf' % (co2_idx)])
            sig_data = sig_data[np.logical_not(np.isnan(sig_time))]
            Sxx,f,t = specgr (sig_data,spec_params)
            all_data[co2_idx]= pd.DataFrame(Sxx.T,columns=['%s_%d' % (co2_idx,x+1) for x in range(np.int32(spec_params['nperseg']/2))],index=t)

        all_data['co2_freq'] = f            

    return all_data


In [14]:
def create_data_profiles_actuators(shotn,timing):
    errflag=0
    all_dfs=dict()
    stats=dict()
    for data_category in ['profiles','shape','pinj','tinj','qmin']:#Categories that can be found in separate files
        hf = h5py.File(main_data_path+'%d_%s.h5' % (shotn,data_category), 'r')

        for sig_name in hf.keys():
            if np.sum(np.asarray(hf[sig_name]['ydata'])) == -1:
                if len(np.asarray(hf[sig_name]['zdata']))<=1:
                    print (print('<<<BAD!>>> %s: 1d signal' % (sig_name)))
                    errflag=1
                    continue
                else:
                    df = pd.DataFrame(np.asarray(hf[sig_name]['zdata']),index=np.asarray(hf[sig_name]['xdata']),columns=[sig_name])
                    df.index.rename('time', inplace=True)
            elif np.asarray(hf[sig_name]['xunits'])==b'ms':
                df = pd.DataFrame(np.asarray(hf[sig_name]['zdata']),index=np.asarray(hf[sig_name]['xdata']),columns=[sig_name+str(x+1) for x in range(len(np.asarray(hf[sig_name]['ydata'])))])
                df.index.rename('time', inplace=True)
            elif np.asarray(hf[sig_name]['yunits'])==b'ms':
                df = pd.DataFrame(np.asarray(hf[sig_name]['zdata']).T,index=np.asarray(hf[sig_name]['ydata']),columns=[sig_name+str(x+1) for x in range(len(np.asarray(hf[sig_name]['xdata'])))])
                df.index.rename('time', inplace=True)
            else:
                print (print('<<<BAD!>>> %s: 2d signal' % (sig_name)))
                errflag=1
            df.drop(df[df.index < timing['begin']].index, inplace=True) #cut everything before given begin time
            df.drop(df[df.index > timing['end']].index, inplace=True)#cut everything after given end time
            all_dfs[sig_name]=df
        hf.close()
        # If only echpwrc is required, do not check the other ech signals
        data_category = 'ech'
        hf = h5py.File(main_data_path+'%d_%s.h5' % (shotn,data_category), 'r')
        sig_name = 'echpwrc'
        if np.sum(np.asarray(hf[sig_name]['ydata'])) == -1:
            if len(np.asarray(hf[sig_name]['zdata']))<=1:
                print (print('<<<BAD!>>> %s: 1d signal' % (sig_name)))
                errflag=1
                continue
            else:
                df = pd.DataFrame(np.asarray(hf[sig_name]['zdata']),index=np.asarray(hf[sig_name]['xdata']),columns=[sig_name])
                df.index.rename('time', inplace=True)
        elif np.asarray(hf[sig_name]['xunits'])==b'ms':
            df = pd.DataFrame(np.asarray(hf[sig_name]['zdata']),index=np.asarray(hf[sig_name]['xdata']),columns=[sig_name+str(x+1) for x in range(len(np.asarray(hf[sig_name]['ydata'])))])
            df.index.rename('time', inplace=True)
        elif np.asarray(hf[sig_name]['yunits'])==b'ms':
            df = pd.DataFrame(np.asarray(hf[sig_name]['zdata']).T,index=np.asarray(hf[sig_name]['ydata']),columns=[sig_name+str(x+1) for x in range(len(np.asarray(hf[sig_name]['xdata'])))])
            df.index.rename('time', inplace=True)
        else:
            print (print('<<<BAD!>>> %s: 2d signal' % (sig_name)))
            errflag=1
        df.drop(df[df.index < timing['begin']].index, inplace=True)
        df.drop(df[df.index > timing['end']].index, inplace=True)
        all_dfs[sig_name]=df
        hf.close()
    return (all_dfs,stats)
            

In [15]:
# The usual spectrogram extractor and enhacement pipeline
def specgr (sig_in,spec_params,thr=0.9, gaussblr_win=(31,3)):
    f, t, Sxx = scipy.signal.spectrogram(sig_in, nperseg=spec_params['nperseg'], noverlap=spec_params['noverlap'],fs=spec_params['fs'], window=spec_params['window'],scaling=spec_params['scaling'], detrend=spec_params['detrend'])
    Sxx = np.log(Sxx + spec_params['eps'])
    Sxx=(Sxx-np.min(Sxx))/(np.max(Sxx)-np.min(Sxx))
    Sxx = Sxx[:-1,:];f=f[:-1]
    if spec_params['enhanced']:
        Sxx= quantfilt(Sxx,thr)
        Sxx =  gaussblr(Sxx,gaussblr_win)
        Sxx = meansub(Sxx)    
        Sxx = morph(Sxx)
        Sxx = meansub(Sxx)    
    return Sxx,f,t*1000

# ---- Spectrogram enahncement begin ----

def norm(data):
    mn = data.mean()
    std = data.std()
    return((data-mn)/std)

def rescale(data):
    return (data-data.min())/(data.max()-data.min())

def quantfilt(src,thr=0.9):
    filt = np.quantile(src,thr,axis=0)
    out = np.where(src<filt,0,src)
    return out

# gaussian filtering
def gaussblr(src,filt=(31, 3)):
    src = (rescale(src)*255).astype('uint8')
    out = cv2.GaussianBlur(src,filt,0)
    return rescale(out)

# mean filtering
def meansub(src):
    mn = np.mean(src,axis=1)[:,np.newaxis]
    out = np.absolute(src - mn)
    return rescale(out)

# morphological filtering
def morph(src):
    src = (rescale(src)*255).astype('uint8')
    se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    mask = cv2.morphologyEx(src, cv2.MORPH_CLOSE, se1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
    return rescale(mask)
# ---- Spectrogram enahncement end ----

In [37]:
shotn = all_shots[100]
timing={'begin':0,
       'end':4000
       }
ece_co2_data= create_data_ece_co2(shotn,timing) # Get ECE-CO2 dataframe
final_df = pd.DataFrame(index=ece_co2_data['ece01'].index) # use the time of first channel of ECE as reference 
for df in [ece_co2_data['ece%.2d' % (idx+1)] for idx in range(40)]+[ece_co2_data[idx] for idx in ['r0','v1','v2','v3']]:
    final_df=pd.merge_asof(final_df, df, left_index=True, right_index=True) #Merge all ECE-CO2 channel based on the time of ECE01

ECE data found!
CO2 data found!


In [38]:
prof_act_data,prof_act_stats = create_data_profiles_actuators(shotn,timing) #Get profile-actuator dataframe
feat_list=[  'bmspinj' #Select which columns to be merged as the final dataframe
            ,'bmstinj'
            ,'pcbcoil'
            ,'iptipp'
            ,'kappa'
            ,'triangularity_u'
            ,'triangularity_l'
            ,'echpwrc'
          ]+['pinjf_%s%s' % (pw,sd) for pw in ['15','21','30','33'] for sd in ['l','r']]+['tinj_%s%s' % (pw,sd) for pw in ['15','21','30','33'] for sd in ['l','r']]
for feat in feat_list:
    final_df=pd.merge_asof(final_df, prof_act_data[feat], left_index=True, right_index=True) # merge the selected columns of profil-actuator to ECE-CO2

pickle.dump(final_df,open('temp.pkl','wb')) # dump the final dataframe